# Quantitative evaluation with a workflow

It is crucial to qualitatively evaluate the performance of anomaly detectors to know their capabilities. For this, ``dtaianomaly`` offers the ``Workflow``: detect anomalies in a large set of time series using various detectors, and to measure their performance using multiple evaluation criteria. The ``Workflow`` facilitates the validation of the anomaly detectors, because you only need to define the different components.

There are two ways to run a ``Workflow`` from Python or from a configuration file.

> You can also evaluate [custom components](https://dtaianomaly.readthedocs.io/en/stable/getting_started/custom_models.html) in ``dtaianomaly`` via a ``Workflow`` in Python. However, this is not possible via a configuration file without extending the functionality of the workflow_from_config function!

In [1]:
from dtaianomaly.workflow import Workflow
from dtaianomaly.data import UCRLoader
from dtaianomaly.evaluation import AreaUnderROC, AreaUnderPR, Precision
from dtaianomaly.thresholding import TopN, FixedCutoff
from dtaianomaly.preprocessing import Identity, ZNormalizer, ChainedPreprocessor, MovingAverage, ExponentialMovingAverage
from dtaianomaly.anomaly_detection import LocalOutlierFactor, IsolationForest

## Run a workflow from Python

Here we will initialize the different components to evaluate in the ``Workflow``. We start by creating a list of ``LazyDataLoader`` objects. We manually selected two time series to use for evaluation, but alternatively you can use all datasets in some directory using the ``from_directory()`` method in the data module. 

In [2]:
dataloaders = [
    UCRLoader('../data/UCR-time-series-anomaly-archive/001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.txt'),
    UCRLoader('../data/UCR-time-series-anomaly-archive/002_UCR_Anomaly_DISTORTED2sddb40_35000_56600_56900.txt')
]

Next, we initialize a number of ``Preprocessor``s. Below, we create 4 preprocessors to analyze the effect of Z-normalization combined with smoothing. We also add the ``Identity`` preprocessor, to analyze what happens if no preprocessing is done. 

In [3]:
preprocessors = [
    Identity(),
    ZNormalizer(),
    ChainedPreprocessor([MovingAverage(10), ZNormalizer()]),
    ChainedPreprocessor([ExponentialMovingAverage(0.8), ZNormalizer()])
]

We will now initialize our anomaly detectors. Each anomaly detector will be combined with each preprocessor, and applied to each time series. 

In [4]:
detectors = [LocalOutlierFactor(50), IsolationForest(50)]

Finally, we need to define the ``Metric``s used to evaluate the models. Both ``BinaryMetric`` and ``ProbaMetric`` can be provided. However, the workflow evaluates the scores obtained by the ``predict_proba`` method of the ``BaseDetector``. To evaluate a ``BinaryMetric``, a number of thresholding strategies must be provided to convert the continuous anomaly probabilities to discrete anomaly labels. Each thresholding strategy is combined with each thresholding metric. The thresholds have no effect on the ``ProbaMetric``s.

> To save on computational resources, the anomaly detector is used once to detect anomalies in a time series, and the predicted anomaly scores are used to evaluate all anomaly scores. This means that there is no computational overhead on providing more metrics, besides the resources required to compute the metric. 

In [5]:
thresholds = [TopN(20), FixedCutoff(0.5)]
metrics = [Precision(), AreaUnderPR(), AreaUnderROC()]

Once all components are defined, we initialize the ``Workflow``. We also define additional parameters, such ``n_jobs``, to allow for multiple anomaly detectors to detect anomalies in parallel. Then, we can execute the workflow by calling the ``run()`` method, which returns a dataframe with the results. 

In [6]:
workflow = Workflow(
    dataloaders=dataloaders,
    metrics=metrics,
    thresholds=thresholds,
    preprocessors=preprocessors,
    detectors=detectors,
    n_jobs=4
)
workflow.run()

,Dataset,Detector,Preprocessor,Runtime [s],TopN(n=20)->Precision(),FixedCutoff(cutoff=0.5)->Precision(),AreaUnderPR(),AreaUnderROC()
0,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),Identity(),12.412205,0.0,0.000000,0.008296,0.415101
1,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),Identity(),2.543002,1.0,0.082974,0.283506,0.711889
2,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),ZNormalizer(),12.455472,0.0,0.000000,0.008296,0.415101
3,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),ZNormalizer(),2.223877,1.0,0.108569,0.292870,0.715387
4,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),MovingAverage(window_size=10)->ZNormalizer(),21.683860,0.0,0.000000,0.010392,0.422901
5,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),MovingAverage(window_size=10)->ZNormalizer(),10.538443,1.0,0.101343,0.284716,0.716597
6,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),ExponentialMovingAverage(alpha=0.8)->ZNormaliz...,13.232668,0.0,0.000000,0.010401,0.432582
7,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),ExponentialMovingAverage(alpha=0.8)->ZNormaliz...,2.048298,1.0,0.103520,0.293610,0.725637
8,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),Identity(),13.419250,0.0,0.000000,0.009152,0.495514
9,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),Identity(),2.367389,0.0,0.000000,0.001949,0.103307


## Run a workflow from a configuration file

Alternatively, you can define a workflow using JSON configuration files. The file [Config.json]() illustrates how the workflow defined above can be written as a configuration file. More details regarding the syntax are provided below. Using the ``workflow_from_config`` method, you can pass the path to a configuration file to create the corresponding ``Workflow``, as shown in the example below. Then, you can run the ``Workflow`` via the ``run()`` function. 
 

In [7]:
from dtaianomaly.workflow import workflow_from_config
workflow = workflow_from_config("Config.json")
workflow.run()

,Dataset,Detector,Preprocessor,Runtime [s],TopN(n=20)->Precision(),FixedCutoff(cutoff=0.5)->Precision(),AreaUnderPR(),AreaUnderROC()
0,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),Identity(),11.796181,0.0,0.000000,0.008296,0.415101
1,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),Identity(),2.066957,1.0,0.104561,0.277147,0.723739
2,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),ZNormalizer(),11.857119,0.0,0.000000,0.008296,0.415101
3,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),ZNormalizer(),2.609010,1.0,0.112093,0.292322,0.715439
4,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),MovingAverage(window_size=10)->ZNormalizer(),19.901729,0.0,0.000000,0.010392,0.422901
5,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),MovingAverage(window_size=10)->ZNormalizer(),10.524450,1.0,0.109982,0.301381,0.720019
6,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),ExponentialMovingAverage(alpha=0.8)->ZNormaliz...,13.658639,0.0,0.000000,0.010401,0.432582
7,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),ExponentialMovingAverage(alpha=0.8)->ZNormaliz...,3.242999,1.0,0.099493,0.292165,0.723519
8,UCRLoader(path=../data/UCR-time-series-anomaly...,LocalOutlierFactor(window_size=50),Identity(),13.380715,0.0,0.000000,0.009152,0.495514
9,UCRLoader(path=../data/UCR-time-series-anomaly...,IsolationForest(window_size=50),Identity(),2.227625,0.0,0.000000,0.001943,0.095716


A configuration file is build from different entries, with each entry representing a component of the ``Workflow``. These entries are build as follows:
```json
    { 'type': <name-of-component>, 'optional-param': <value-optional-parameter>}
```
The ``'type'`` equals the name of the component, for example ``'LocalOutlierFactor'`` or ``'ZNormalizer'``. This string must exactly match the object name of the component you want to add to the workflow. In addition, it is possible to define hyperparameters of the component. For example for ``'LocalOutlierFactor'``, you must define a
``'window_size'``, but can optionally also define a ``'stride'``. An error will be raised if the entry has missing obligated parameters or unknown parameters.

The configuration file itself is also a dictionary, in JSON format. The keys of this dictionary correspond to the parameters of the ``Workflow``. The corresponding values can be either a single entry (if one component is requested) or a list of entries (if multiple components are requested).

Below, we show a simplified version of the configuration in [Config.json](). 

```json
{
  "dataloaders": {
    "type": "UCRLoader",
    "path":"../data/UCR-time-series-anomaly-archive/001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.txt"
  },
  "metrics": [{"type": "Precision"}, {"type": "AreaUnderPR"}],
  "thresholds": {"type": "FixedCutoff", "cutoff": 0.5},
  "preprocessors": {"type": "ZNormalizer"},
  "detectors": [
    {"type": "LocalOutlierFactor", "window_size": 50},
    {"type": "IsolationForest", "window_size": 50}
  ],
  "n_jobs": 4
}
```